# Metageomics


# Introduction

The human microbiome is involved in many essential functions, such as food digestion and modulation if immune system. Many diseases have been associated with microbiota disbiosis among them, psoriasis and chorinc skin inflammation reguarding skin flora alteration, and colorectal cancers and  inflammatory bowel deseases, in association with colonic microbiota disbiosis. High troughtput DNA sequencing tecniques allowed to study the entire genome of a microbiotic population and also quantify its aboundance. 

The main steps of a microbila study are:
<ol>
     <li>Mirobial DNA extraction and sequencing, using two approches, amplicon sequencing (16S) or shotgun sequencing</li>
    <li>Bioinformatic sequence processing</li>
    <li>Statistical analysis</li>
</ol>
Amplicon sequencing relies on, after a PCR amplification, sequencing a phylogenetic marker gene, which is 16S ribosomal RNA gene, that encodes the small ribosomal subunit for the archea and bacteria.
The 16S gene contains both high conserved areas and hypervariable sites, denoted as V1-V9.
The conserved regions can be targeted with PCR primers, while hypervariable sites are specific for each microbial species(or OTU), which allows to dicriminate different microbes.
V1-V3 or V4 are usually used as target in PCR amplification, from which thousands to millions of DNA copies are generated. PCR's amplicons are than sequenced by hightroughput tecniques, and multiples nucleotide sequenced called reads are obtained. 

The most used bioinformatic pipelines in amplicon sequencing are: mothur and <b>QIIME</b>. 
Bioinformatic processing consist in five steps:

<ol>
    <li>Preprocessing and quality control filtering</li>
<li>OTU(operation taxonomic unit) binning </li>
<li>Taxonomy assignment </li>
<li>Construction of abundance table </li>
<li>Phylogenetic analysis </li>
</ol>

Preprocessing and quality control filtering consists in assign sequences to samples, using the barcode contained in each sequence (<b>Demultiplexing</b>), then sequences are quality filtered to remove too short sequences, too many ambiguos base pairs and chimeras. 

OTU binning is the process of clustering similar DNA sequences into OTUs (groups of DNA sequnces with at least 97% similarity). The sequences assigned to OTUs are representative sequences determined by the most common nucleotide founded in each position.

Taxonomic assignment consists in comparing OTU consensus sequences to microbial 16S rRNA refrence database such as Green Genes, SILVA or RDP. This step provides the available annotation of each OTU to the different taxonomy level. Many OTUs are not annoted expecially for lowest taxonomic levels. 

Each entry of the OTU abundance table contains the number of reads observed for each OTU for each sample.
OTU table can be very sparse, if there are too many zeros(many OTUs observed only in a few samples) could be useful to aggomerate OTUs at a broader taxonomic level. 

Phylogenetic trees can be used to obtein phylogeetic distances between samples.

Beacause of the experimental process and the quality control filtering, microbiome data are very noisy and the total number of reads for sample is vary variable, which requires normalization of data before comparing the abundance between samples. The maximum number of sequences reads that the DNA sequencer can provide costraints the total number of counts for sample. It can bias the real abunances of each taxonomic unit. 

Statistical analysis starts with normalization and procedes with an exploratory part which consists in the analisys of diversity measures and their visualization thought multivariate techniques to visualize species abundances in a low dimentional space.


## QIIME2

All data must be imported as a <b>QIIME2 artifact</b> to be used by a QIIME2 action (with the exception of some metadata). All amplicon/metagenome sequencing experiments begin, at some point or another, as raw sequence data. This is probably FASTQ data, containing DNA sequences and quality scores for each base.
We must demultiplex these reads to determine which sample each read came from.
Reads should then be denoised into amplicon sequence variants (ASVs) or clustered into operational taxonomic units (OTUs) to achieve two goals:

<ol>
    <li>reducing sequence errors</li>

   <li> dereplicated sequences</li>
</ol>

The resulting feature table and representative sequences are key pieces of data. A feature table is essentially a matrix of samples x observations, i.e., the number of times each “feature” (OTUs, ASVs, etc) is observed in each sample in a data set.
We can do many things with this feature table. Common analyses include:
Taxonomic classification of sequences (a.k.a., “what species are present?”)
Alpha and beta diversity analyses, or measures of diversity within and between samples, respectively (a.k.a., “how similar are my samples?”)
Many diversity analyses rely on the phylogenetic similarity between individual features. If you are sequencing phylogenetic markers (e.g., 16S rRNA genes), you can align these sequences to assess the phylogenetic relationship between each of your features.
Differential abundance measurements determine which features (OTUs, ASVs, taxa, etc) are significantly more/less abundant in different experimental groups.

<img src="https://docs.qiime2.org/2019.10/_images/overview.png">

### Demultiplexing
Most nextgen sequencing instruments have the capacity to analyze hundreds or even thousands of samples in a single lane/run; we do so by multiplexing these samples, which is just a fancy word for mixing a whole bunch of stuff together. How do we know which sample each read came from? This is typically done by appending a unique barcode (index or tag) sequence to one or both ends of each sequence. Detecting these barcode sequences and mapping them back to the samples they belong to allows us to demultiplex our sequences.
You should know which barcode belongs to each sample and include this barcode information in your sample metadata file. Paired-end reads need to be joined at some point in the analysis, this happens automatically during denoising with q2-dada2.


### Denoising and clustering

The names for these steps are very descriptive:
We denoise our sequences to remove and/or correct noisy reads.
It dereplicates input sequences to reduce repetition and file size/memory requirements in downstream steps (it keeps count of each replicate).
It clusters sequences to collapse similar sequences (e.g., those that are ≥ 97% similar to each other) into single replicate sequences. This process, also known as <b>OTU picking</b>, was once a common procedure, used to simultaneously dereplicate but also perform a sort of quick-and-dirty denoising procedure (to capture stochastic sequencing and PCR errors, which should be rare and similar to more abundant centroid sequences). Use denoising methods instead if you can.

### Denoising (ASVs)

The denoising methods currently available in QIIME 2 include DADA2 and Deblur. You can learn more about those methods by reading the original publications for each. Examples of DADA2 exist in the moving pictures tutorial and Fecal Microbiome Transplant study tutorial (for single-end data) and Atacama soils tutorial (for paired-end data). Examples of Deblur exist in the moving pictures tutorial (for single-end data) and read joining tutorial (for paired-end data). Note that deblur (and also vsearch dereplicate-sequences) should be preceded by basic quality-score-based filtering, but this is unnecessary for dada2. Both Deblur and DADA2 contain internal chimera checking methods and abundance filtering, so additional filtering should not be necessary following these methods.

To put it simply, these methods filter out noisy sequences, correct errors in marginal sequences (in the case of DADA2), remove chimeric sequences, remove singletons, join denoised paired-end reads (in the case of DADA2), and then dereplicate those sequences. The products of these operations are called <b>ASVs</b>.


### Clustering (OTU)
Next we will discuss clustering methods. Dereplication (the simplest clustering method, effectively producing 100% OTUs, i.e., all unique sequences observed in the dataset) and it is the necessary starting point to all other clustering methods in QIIME2.
<b>q2-vsearch</b> implements three different OTU clustering strategies: de novo, closed reference, and open reference. 

All should be preceded by basic quality-score-based filtering and followed by chimera filtering and aggressive OTU filtering (the treacherous trio, a.k.a. the Bokulich method).

The features produced by clustering methods are known as operational taxonomic units (OTUs), which is Esperanto for suboptimal, imprecise rubbish. 

While OTU clustering approaches attempt to blur similar sequences into an abstracted consensus sequence, thus minimizing the influence of any sequencing errors within the pool of reads, the Amplicon Sequence Variant (ASV) approach attempts to go the opposite direction. The ASV approach will start by determining which exact sequences were read and how many times each exact sequence was read. These data will be combined with an error model for the sequencing run, enabling the comparison of similar reads to determine the probability that a given read at a given frequency is not due to sequencer error. This creates, in essence, a p-value for each exact sequence, where the null-hypothesis is equivalent to that exact sequence being a consequence of sequencing error.

### The Feature Table

The final products of all denoising and clustering methods/workflows are a FeatureTable(Frequency feature table) artifact and a FeatureData(representative sequences) artifact. These are two of the most important artifacts in an amplicon sequencing workflow, and are used for many downstream analyses, as discussed below. Indeed, feature tables are crucial to any QIIME2 analysis, as the central record of all observations per sample. Such an important artifact deserves its own powerful plugin, q2-feature-table.

### Taxonomy classification and taxonomic analyses

It can be done by comparing our query sequences (i.e., our features, be they ASVs or OTUs) to a reference database of sequences with known taxonomic composition. Simply finding the closest alignment is not really good enough — because other sequences that are equally close matches or nearly as close may have different taxonomic annotations. 

So we use taxonomy classifiers to determine the closest taxonomic affiliation with some degree of confidence or consensus (which may not be a species name if one cannot be predicted with certainty!), based on alignment, k-mer frequencies, etc.

## Microbiome Compositional Data 

Microbiome data is compositional because the information that abundance tables contain is relative. In a microbiome abundance table, the total number of counts per sample is highly variable and constrained by the maximum number of DNA reads that the sequencer can provide. This total count constraint induces strong dependencies among the abundances of the different taxa; an increase in the abundance of one taxon implies the decrease of the observed number of counts for some of the other taxa so that the total number of counts does not exceed the specified sequencing depth. Moreover, observed raw abundances and the total number of reads per sample are non-informative since they represent only a fraction or random sample of the original DNA content in the environment. These characteristics of microbiome abundance data clearly fall into the notion of compositional data.

In [1]:
library('qiime2R')
library('vegan')
library('ggplot2')
library('plotly')
library('DESeq2')
library('phyloseq')
library('reshape2')
library('plyr')

Loading required package: permute

Loading required package: lattice

This is vegan 2.5-6


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout


Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colMeans,
    colnames, colSums, dirname, do.cal

In [5]:
#setwd('/home/adelaide/adelaide/Dottorato/analisi/gastroDataAnalisi/')

source("function.R")

#definizione dei path dei file di output di QIIME2
Tab <- 'DADA2_output/dada2_gastrodata_table.qza'
Map  <- 'Metadata/Mapping.txt'
Tree <- 'Phylogenetic_tree/rooted_tree.qza'
Tax <- "Taxonomic_assignment/taxonomy_classification_data_sklearn.qza"



# crea l'oggetto phyloseq 
phy <- qza_to_phyloseq(Tab, Tree, Tax, Map, tmp='Temp')

#
phy <- subset_samples( phy, IBD=="0" | IBD=="1")#  | IBD=="2" )

#corregge l'importazione della tassonomia silva
phy <- importaTassonomiaSilva(phy)


# trasforma IBD (e ID_original) in un factor, per i grafici
metadata <- sample_data(phy)
metadata$IBD <- as.factor(metadata$IBD)
metadata$ID_original <- as.factor(metadata$ID_original)
sample_data(phy) <- metadata

#agglomerazione per...
phyF <-tax_glom(phy,"Family")

phyG <-tax_glom(phy,"Genus")

# trasforma in dati percentuali
phyF_percentuali <- transform_sample_counts(phyF, function(x) {x/sum(x)*100} )
phyG_percentuali <- transform_sample_counts(phyG, function(x) {x/sum(x)*100} )
# prende la tassonomia (trasformata in matrice, poi in dataframe (altrimenti non funzionava) ) per ricavare gli ID dei clostridi
tassonomiaF <- as(tax_table(phyF_percentuali),"matrix")
tassonomiaF <- as.data.frame(tassonomiaF)

tassonomiaG <- as(tax_table(phyG_percentuali),"matrix")
tassonomiaG <- as.data.frame(tassonomiaG)

Warning message:
“Expected 7 pieces. Missing pieces filled with `NA` in 6755 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


In [3]:
colnames(sample_data(phyG))

[1] "BarcodeSequence"                          
 [2] "LinkerPrimerSequence"                     
 [3] "ID_F"                                     
 [4] "ID_original"                              
 [5] "Data_prelievo"                            
 [6] "Prelievo"                                 
 [7] "Nato"                                     
 [8] "Eta"                                      
 [9] "Genere"                                   
[10] "Fumo"                                     
[11] "Familiarità_IBD_"                         
[12] "IBD"                                      
[13] "Montreal_RCU_localizzazione"              
[14] "Montreal_MDC_Eta"                         
[15] "Montreal_MDC_localizzazione"              
[16] "Montreal_MDC_Fenotipo"                    
[17] "Malattia_perianale"                       
[18] "Terapia_biologica_al_momento_del_prelievo"
[19] "Immunosoppressore_al_momento_del_prelievo"
[20] "Altra_terapia_al_momento_del_prelievo"    
[21] "Pregressa_tp_biologica"                   
[22] "Pregressa_tp_con_immunosoppressori"       
[23] "Interventi_chirurgici_per_malattia"       
[24] "Attività"                                 
[25] "Manifestazioni_extraintestinali"          
[26] "MEI_attive"                               
[27] "Tipo_MEI"                                 
[28] "Calprotectina_.250_.mg.kg."               
[29] "PCR_.mg.L."                               
[30] "Emoglobina_.gr.dl."                       
[31] "Globuli_bianchi_.mila.microL."            
[32] "Colesterolo_totale_.mg.dl.__"             
[33] "Colesterolo_LDL_.mg.dl."                  
[34] "Colesterolo_HDL_.mg.dl."                  
[35] "Trigliceridi_.mg.dl.__"                   
[36] "Glicemia_.mg.dl."                         
[37] "Steatosi_epatica"                         
[38] "Attività_endoscopica_.SES_CD._Mayo."      
[39] "Sicus"                                    
[40] "Peso"                                     
[41] "Altezza"                                  
[42] "Parto"                                    
[43] "Calo_ponderale"                           
[44] "Aumento_peso"                             
[45] "Yogurt"                                   
[46] "Pane"                                     
[47] "Pasta"                                    
[48] "Latticini"                                
[49] "Frutta.Verdua"                            
[50] "Carne"                                    
[51] "Pesce"                                    
[52] "Cereali"                                  
[53] "Legumi"                                   
[54] "Caffè"                                    
[55] "Alcool"                                   
[56] "Pizza"                                    
[57] "Fumo_al_momento_del_prelievo"             
[58] "Esercizio_Fisico"                         
[59] "Comorbidità_degne_di_nota"                
[60] "Comorbidità_prichiatriche"

In [4]:
#as.data.frame(tax_table(phy))[as.data.frame(tax_table(phy))$Order == "Clostridiales",]

In [5]:
'''#agglomerazione per genere
phyG <-tax_glom(phy,"Genus")

mat <- t(cbind(as(otu_table(phyG), "matrix"), as(tax_table(phyG)[,"Genus"], "matrix")))
mat <- as.data.frame(mat)
ID <- append(c(sample_data(phyG)$ID_original),"NA")
IBD <- append(c(sample_data(phyG)$IBD),"NA")
mat$ID <- ID
mat$IBD <- IBD

mat$IBD <- revalue(mat$IBD, c("2"="1", "1"="0"))

write.csv(t(mat),"reads4Genus.csv")

mat <- as.data.frame(t(mat))

clost <- mat[c(which(grepl('Clostridium',mat$Genus,fixed=FALSE)),279,280),]
clost_ <- mat[c(which(grepl('clostridium',mat$Genus,fixed=FALSE)),279,280),]

clostridium <- rbind(clost[1:5,],clost_)

clostridium <- as.data.frame(t(clostridium))
clostridium$IBD <- revalue(clostridium$IBD, c("2"="1", "1"="0"))

write.csv(clostridium,"reads4Genus_clostridium.csv")

sel <- c("21","54" ,"59","63","66","69","72","83","87","91","97","98","101","102","107","1","9","9","15","19","22","24","25","37","38","42","51","52","63","62","65","68")

antony <- clostridium %>% filter( ID %in% sel)
traspose <- as.data.frame(t(clostridium))
antony <- as.data.frame(t(antony))
antony$Genus <- traspose$Genus

#write.csv(antony,"reads4Genus_clostridium_antonio.csv") '''

ERROR: Error in parse(text = x, srcfile = src): <text>:1:3: unexpected string constant
16: 
17: clost <- mat[c(which(grepl('
      ^


# Normalization

The large variability of the total counts per sample prevents meaningful comparisons of raw abundances between individuals, therefore it is needed to normalize data before any statistical analysis.
There are different method to normalize:

<ul>
    <li> Computation of relative abundances by dividing the raw abundance by the total number of counts per sample </li>
    <li> Rarefaction, which consists on subsampling the same number of reads for each sample, so that every samples have the same number of total counts ( not raccomanded beacause it leads to a loss of information. </li>
    <li> Deseq2 normalization tecnique, which is the most realiable. </li>

## Rarefaction

In [ ]:
### normalizzazione con rarefazione

phy_rar <- rarefy_even_depth(phyS, sample.size=6000, rngseed=1)
##
G1 <- testPermanovaPCoA(phy_rar,DistancePhyl="bray",Variable="IBD",Permutations=9999)
G2 <- testPermanovaPCoA(phy_rar,DistancePhyl="unifrac",Variable="IBD",Permutations=9999)
G3 <- testPermanovaPCoA(phy_rar,DistancePhyl="wunifrac",Variable="IBD",Permutations=9999)
G4 <- testPermanovaPCoA(phy_rar,DistancePhyl="canberra",Variable="IBD",Permutations=9999)
##
library("ggpubr")
P <- ggarrange(G1,G2,G3,G4,ncol=2,nrow=2,labels=c("A","B","C","D"))
##
#Save.graph(P, OutDir="Immagini" , Name="PCoA_Silva_Rarefazione_IBD" , Name_prefix="001")
P

## Deseq2

High-throughput microbiome datasets generated by the 16S ribosome RNA (rRNA) gene sequencing or shotgun metagenomic sequencing have some properties that require tailored analytic tools; these include count compositional structure, <b>varied total sequence reads across samples</b>, <b>over-dispersion</b> and <b>zero-inflation</b>. 

One way to account for varing total reads is normalization, i.e., conversion of the sequence counts to the relative abundance (or proportion) using the total sum, mean, or median of representative OTUs across all samples. The negative binomial regression, which is a standard statistical method for analyzing over-dispersed count observations, has been recently applied to microbiome data. On the other hand, several zero-inflated models have also been proposed to correct for excess zero counts in microbiome measurements, including zero-inflated Gaussian, lognormal, negative bimomial and beta models.

The <b>negative binomial distribution</b> also arises as a continuous mixture of Poisson distributions (i.e. a compound probability distribution) where the mixing distribution of the Poisson rate is a gamma distribution. Read counts for a sample in theory follow a Binomial(n,p) distribution, where n is the total number of reads and p is the probability of a read mapping to a specific gene. However, the binomial distribution is computationally inconvenient, since it involves computing factorials, and with millions of reads in each sample, the Poisson distribution (with lambda = n*p) is an excellent approximation to the Binomial while being far more mathematically tractable. So the Poisson noise quantifies the counting uncertainty, while the gamma distribution quantifies the variation in gene expression between replicates. The mixture of the two yields the negative binomial.
The negative binomial is a 

DESeq2’s median of ratios consists in dividing counts by sample-specific size factors determined by median ratio of gene counts relative to geometric mean per gene (OTU). This is performed by dividing each raw count value in a given sample by that sample’s normalization factor to generate normalized count values. This is performed for all count values (every gene in every sample). Log-ratio approach has been introduced by Aitchison and laid the foundations of Compositional Data Analysis (CoDA). Mathematically, the assertion that the relevant information is contained in the ratios between the components implies that two proportional compositions are equally informative and this induces equivalence classes of vectors carrying the same information. In microbiome analysis, for example, both the raw counts and their transformation into relative abundances or proportions belong to the same equivalence class and they carry the same relative information.

<img src="Istantanea_2020-02-18_10-16-12.png">

In [3]:
#otu_table(phy) <- otu_table(phy)

#trasforma l'oggetto phyloseq in un oggetto deseq2
Q <- phyloseq_to_deseq2(phyF, ~ IBD)

#calcolo della media geometrica
gm_mean <- function(x, na.rm=TRUE){
    exp(sum(log(x[x>0]),na.rm=na.rm) / length(x) )
}

# normalizza usando DESeq2, applicando la media geometrica ai dati
geoMeans = apply(counts(Q),1,gm_mean)

#applica un fattore di correzione
# le OTU con un numero di reads molto alto possono portare a sottostimare 
#l'abbondanza di altre OTU con un numero di reads più basso
D <- estimateSizeFactors(Q, geoMeans = geoMeans)

normalized_count <- as.data.frame(round(counts(D,normalized=TRUE)))

otu_table(phyF) <- otu_table(normalized_count, taxa_are_rows=TRUE)

# salva in un dataframe per poterli poi ricaricare
#write.table(normalized_count,"DADA2_output/deseq2_otu_normalized.txt",sep="\t", quote=FALSE)

converting counts to integer mode



## Diversity analisys

The diversity of the microbiome is an important indicator of the good or bad conditions of the ecosystem, with larger microbiome diversity being usually associated to better health status. Microbiome diversity can be assessed through multiple ecological indices that can be divided into two kind of measures:
<ol>
    <li><b>Alpha diversity</b> which measures the variability of species within a sample. </li>
    <li><b>Beta diversity</b> which accounts for the differences in composition between samples.</li>
</ol>
The R package <b>vegan</b> provides a large set of diversity measures.

### Alpha diversity

Alpha diversity within sample diversity The most important measure of alpha diversity is <b>richness</b>, defined as the number of different species present in an environment. Richness is estimated by the observed richness, the number of different species observed in the sample. The observed richness tends to underestimate the real richness in the environment, where the less frequent species are likely to be undetected. There are different indices that adjust for this and try to estimate the hidden part that has not been detected. One of the most extended richness measure is <b>Chao1 index</b>. 

Another important indicator of alpha diversity is <b>evenness</b>, which measures the homogeneity in abundance of the different species in a sample. A commonly used measure of evenness is the <b>Shannon index</b>.

### Beta diversity

Beta diversity measures the differences in microbiome composition between samples. There is a wide range of ecological distances or dissimilarities for measuring how close are two microbial compositions. The most commonly used are <b>Bray-Curtis</b>, <b>UniFrac</b> and <b>weighted UniFrac</b> distances. We also define the Aitchison distance which is a proper distance for compositional data.
<ul>
<b>UniFrac</b> family of distances consider the phylogenetic tree that represents the evolutionary relationships among the different taxa. The phylogenetic tree can be obtained from the bioinformatic pipelines, such as mothur and <b>QIIME</b>. For a tree with r branches, let b = (b<sub>1</sub>,…,b<sub>r</sub>) represent the length of the different branches in the phylogenetic tree, and q<sub>1</sub> = (q<sub>11</sub> , … ,q<sub>1r</sub> ), and q<sub>2</sub> = (q<sub>21</sub> ,… ,q<sub>2r</sub> ) the relative abundances associated to each branch for the first and the second sample, respectively.
<li><b>Unweighted UniFrac</b> distance measures the relative length of those branches that lead exclusively to species present in only one of the two samples with respect to the total length of all branches in the tree. It only takes into account the presence or absence of the taxa.
</li>
<li><b>Weighted UniFrac</b> distance instead includes information on the relative abundance of each taxa.
</li>
</ul>

### Ordination 

The goal of ordination plots is the visualization of beta diversity for identification of possible data structures. The multidimensional data is represented into a reduced number of orthogonal axes while keeping the main trends of the data and preserving the distances among samples as much as possible. Most commonly used ordination methods for microbiome data are <b>principal coordinates analysis (PCoA)</b>, also known as multidimensional scaling, and non-metric multidimensional scaling (NMDS). 

PCoA an extension of Principal Components Analysis (PCA). Given a distance or dissimilarity matrix, D, PCoA performs eigenvalue decomposition of D<sub>c</sub>'D<sub>c</sub> where D<sub>c</sub> is the centered distance matrix. When D is the Euclidean distance, PCoA results exactly the same as PCA. Care must be taken with PCoA if the selected distance is not metric, because some eigenvalues may be negative and then, the graphical representation will not perform properly. In order to avoid this problem NMDS is more commonly used. Also based on a distance matrix D, NMDS maximizes the rank-based correlation between the original distances and the distances between samples in the new reduced ordination space. The procedure starts with a random configuration and the optimal representation is obtained following an iterative procedure that at each steps improves the rank correlation. 

Ordination plots can be obtained with the R and Bioconductor packages vegan and phyloseq, among others. 

## Microbiome Statistical Inference

### Multivariate differential abundance testing 

Multivariate differential abundance testing refers to a <b>global test of differences in microbial composition</b> between two or more groups of samples. We can distinguish between distance-based or model-based approaches:

<ol>
<li> The most widely used distance-based method for multivariate community analysis is PERMANOVA(Permutational Multivariate Analysis of Variance Using Distance Matrices). The null hypothesis of no differences in composition among groups is formulated by the condition that the different groups of samples have the same center of masses.
    
Implemented in the function <b>"adonis"</b> of the vegan R package, it consists of a multivariate ANOVA based on dissimilarities. The variability within groups is compared against the variability between groups with the usual ANOVA F statistic, but partition of sums-of-squares is applied directly to dissimilarities. 
Significance is evaluated through permutations to generate a distribution of the pseudo F statistic under the null hypotesis. A related and popular distance-based approach is the analysis of similarities, implemented in the function “anosim” of the vegan R package.
</li>
<p>
<li>An interesting model-based approach for multivariate microbiome analysis is Kernel machine regression (KMR), that extends PERMANOVA to a regression framework. KMR is a semi-parametric regression model that includes a nonparametric component. The model can be expressed as a semiparametric linear regression model when the response variable is continuous or as a semiparametric logistic regression model for a dichotomous response variable.
The non-parametric component is related to a Kernel matrix that is a transformation of the distance matrix D of pairwise distances between individuals. </li>
</ol>
</p>

## PERMANOVA

Permutational multivariate analysis of variance (PERMANOVA), is a <u>non-parametric multivariate statistical test</u>. PERMANOVA is used to compare groups of objects and test the null hypothesis that the centroids and dispersion of the groups as defined by measure space are equivalent for all groups. A rejection of the null hypothesis means that either the centroid and/or the spread of the objects is different between the groups. Hence the test is based on the prior calculation of the distance between any two objects included to the experiment. PERMANOVA shares some resemblance to ANOVA where they both measure the sum-of-squares within and between group and make use of F test to compare within-group to between-group variance. However, while ANOVA bases the significance of the result on assumption of normality, PERMANOVA draws tests for significance by comparing the actual F test result to that gained from random permutations of the objects between the groups. Moreover, while PERMANOVA tests for similarity based on a chosen distance measure, ANOVA tests for similarity of the group averages.

In the simple case of a single factor with p groups and n objects in each group,the total sum-of-squares is determined as:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/cfac536edd1016aa559d1ccf78e48679a67c2e04">

where N is the total number of objects, and d<sub>ij</sub> is the squared distance between objects i and j.

Similarly, the within groups sum-of-squares is determined:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/c2d26a665b1f60b98299b888b56e46d4bf3dbfe9">

where  ε<sub>ij</sub> takes the value of 1 if observation i and observation j are in the same group, otherwise it takes the value of zero. Then, the between groups sum-of-squares <i>SS<sub>A</sub></i> can be calculated as the difference between the overall and the within groups sum-of-squares:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/da0a2898b61f40fb4bf77787edcae82c119c35f5">

Finally, a <b>pseudo F-statistic</b> is calculated:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/2399314b3c10e2b1dc24bfe4beda2d665d1f8e14">
where p is the number of groups.

Finally, the PERMANOVA procedure draws significance for the actual F statistic by performing multiple permutations of the data. In each such the items are shuffled between groups. For each such permutation of the data the permutation F statistic is calculated. 

The <b>p-value</b> is then calculated by:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e48ec1eb507b40fd2804f7cd1112b7c7d767f0c2">

Where <i>F</i> is the F statistic obtained from the original data and <i>F<sup>p</sup></i> is a permutation F statistic.

In [ ]:
#normalized_count <- read.table("DADA2_output/deseq2_otu_normalized.txt",sep="\t")

colnames(normalized_count) <- gsub("\\.","-",colnames(normalized_count))
otu_table(phyF) <- otu_table(normalized_count, taxa_are_rows=TRUE)


G1 <- testPermanovaPCoA(phyF,DistancePhyl="bray",method="NMDS",Variable="IBD",Permutations=9999)
G2 <- testPermanovaPCoA(phyF,DistancePhyl="unifrac",method="NMDS",Variable="IBD",Permutations=9999)
G3 <- testPermanovaPCoA(phyF,DistancePhyl="wunifrac",method="NMDS",Variable="IBD",Permutations=9999)
G4 <- testPermanovaPCoA(phyF,DistancePhyl="canberra",method="NMDS",Variable="IBD",Permutations=9999)
G5 <- testPermanovaPCoA(phyF,DistancePhyl="euclidean",method="NMDS",Variable="IBD",Permutations=9999)

library("ggpubr")
P <- ggarrange(G1,G2,G3,G4,ncol=2,nrow=2,labels=c("A","B","C","D"))


Save.graph(P, OutDir="Immagini" , Name="NMDS_Silva_NormDESeq2_IBD_Family" , Name_prefix="001")

P

In [ ]:
#normalized_count <- read.table("DADA2_output/deseq2_otu_normalized.txt",sep="\t")

colnames(normalized_count) <- gsub("\\.","-",colnames(normalized_count))
otu_table(phyF) <- otu_table(normalized_count, taxa_are_rows=TRUE)


G1 <- testPermanovaPCoA(phyF,DistancePhyl="bray",method="PCoA",Variable="IBD",Permutations=9999)
G2 <- testPermanovaPCoA(phyF,DistancePhyl="unifrac",method="PCoA",Variable="IBD",Permutations=9999)
G3 <- testPermanovaPCoA(phyF,DistancePhyl="wunifrac",method="PCoA",Variable="IBD",Permutations=9999)
G4 <- testPermanovaPCoA(phyF,DistancePhyl="canberra",method="PCoA",Variable="IBD",Permutations=9999)
G5 <- testPermanovaPCoA(phyF,DistancePhyl="euclidean",method="PCoA",Variable="IBD",Permutations=9999)

library("ggpubr")
P <- ggarrange(G1,G2,G3,G4,ncol=2,nrow=2,labels=c("A","B","C","D"))


Save.graph(P, OutDir="Immagini" , Name="PCoA_Silva_NormDESeq2_IBD_Family" , Name_prefix="001")

P

In [6]:
metadata <- as(sample_data(phyF),"data.frame")


D_bray <- distance(phyF,method="bray")
D_Uuni <- distance(phyF,method="unifrac")
D_Wuni <- distance(phyF,method="wunifrac")
D_C <- distance(phyF,method="canberra")

distance_matrix <- "D_bray"
Formula <- "IBD+Genere+Eta+Yogurt+Pasta+Pane+Latticini+FruttaVerdua+Carne+Cereali+Legumi+Alcool+Esercizio_Fisico+Fumo_al_momento_del_prelievo"
PERMANOVA_stepwise_backward(metadata,distance_matrix,parameters=Formula)

distance_matrix <- "D_Uuni"
Formula <- "IBD+Genere+Eta+Yogurt+Pasta+Pane+Latticini+FruttaVerdua+Carne+Cereali+Legumi+Alcool+Esercizio_Fisico+Fumo_al_momento_del_prelievo"
PERMANOVA_stepwise_backward(metadata,distance_matrix,parameters=Formula)

distance_matrix <- "D_Wuni"
Formula <- "IBD+Genere+Eta+Yogurt+Pasta+Pane+Latticini+FruttaVerdua+Carne+Cereali+Legumi+Alcool+Esercizio_Fisico+Fumo_al_momento_del_prelievo"
PERMANOVA_stepwise_backward(metadata,distance_matrix,parameters=Formula)

distance_matrix <- "D_C"
Formula <- "IBD+Genere+Eta+Yogurt+Pasta+Pane+Latticini+FruttaVerdua+Carne+Cereali+Legumi+Alcool+Esercizio_Fisico+Fumo_al_momento_del_prelievo"
PERMANOVA_stepwise_backward(metadata,distance_matrix,parameters=Formula)




[1] "D_bray  ~  IBD+Genere+Eta+Yogurt+Pasta+Pane+Latticini+FruttaVerdua+Carne+Cereali+Legumi+Alcool+Esercizio_Fisico+Fumo_al_momento_del_prelievo"


ERROR: Error in qr.fitted(qrhs, G): 'qr' and 'y' must have the same number of rows


## Univariate differential abundance testing 

When significant global differences in microbiome composition are detected between groups of samples, a natural question arises: which particular taxa are responsible of that global difference? 

A common strategy to answer this question is to test every taxa separately for association with the response variable. When the response variable is dichotomous this is known as <b>univariate differential abundance testing</b>. Below are described both, classical and CoDA approaches for univariate differential abundance testing. However, we advise that classical univariate approaches are notably affected by the compositional structure of microbiome data and their results, with large false discovery rates, might be questioned.

Nonparametric tests, like the <b>Wilcoxon rank-sum test</b> or the <b>Kruskal-Wallis test</b>, can be applied. However, more powerful parametric approaches are available, such as the Bioconductor packages edgeR and DESeq2, initially proposed for transcriptomics analysis (RNA-Seq data). 

<u>Both fit a generalized linear model and assume that read counts follow a Negative Binomial distribution</u>. The NB distribution extends the Poisson distribution by allowing the variance to be different from the mean. edgeR and DESeq2 mainly differ in the way they normalize the data.<u> DESeq2 uses size factors that account for differences in sequencing depth between samples and shrinkage for large variances correction</u>. edgeR can be implemented with different normalization methods but the most recommended is TMM. Two CoDA methods that explicitly accounts for the compositional nature of microbiome data are <b>ANCOM</b> and <b>ALDEx2</b>. 

In [ ]:
#level <- "Genus"
level <- "Family"
#phyLevel <-tax_glom(phy,level)
#phyLevel <- phyG
phyLevel <- phyF

Q <- phyloseq_to_deseq2(phyLevel, ~ IBD)

#calcolo della media geometrica
gm_mean <- function(x, na.rm=TRUE){
    exp(sum(log(x[x>0]),na.rm=na.rm) / length(x) )
}

# normalizza usando DESeq2, applicando la media geometrica ai dati
geoMeans = apply(counts(Q),1,gm_mean)

#applica un fattore di correzione
# le OTU con un numero di reads molto alto possono portare a sottostimare 
#l'abbondanza di altre OTU con un numero di reads più basso
D <- estimateSizeFactors(Q, geoMeans = geoMeans)

normalized_count <- as.data.frame(round(counts(D,normalized=TRUE)))

otu_table(phyLevel) <- otu_table(normalized_count, taxa_are_rows=TRUE)


dds <- DESeq(D)
res <- results(dds)

res <- as.data.frame(res)

res <- res[order(res$padj),]

alpha = 0.05
sigtab = res[which(res$padj < alpha), ]
#sigtab = res[which(res$log2FoldChange < -2), ]
sigtab = cbind(as(sigtab, "data.frame"), as(tax_table(phyLevel)[rownames(sigtab), ], "matrix"))
sigtab <- sigtab[order(sigtab$padj),]
table <-sigtab[,c("log2FoldChange","padj",level)]

In [ ]:
table

In [ ]:
table <- table[table$Family != "uncultured" , ]
table <- table[table$Family !="gut metagenome",]

row.names(table) <- table$Family
table

In [ ]:
barplotting_variables<- function(genus,tassonomia,level,phy_percent){
    for (r in 1:nrow(tassonomia)){

	taxa <- as.character(tassonomia[r,]$Family)
	# rimuove il punto che da problemi quando si salva il grafico
	taxa <- gsub('\\.', '', taxa)
	taxa <- gsub(':', '', taxa)
    #print(type(taxa))
	ID_taxa <- rownames(tassonomia[r,])
	cat("Analisi della tassonomia: ",taxa," con ID: ",ID_taxa,"\n")
	
	# analisi per valutare la concentrazione tra IBD = 0 vs IBD = 1
	phy_percentuali_reduced <- take_taxa(phy_percent, ID_taxa)
	#new_order <- c("21","54","59","63","66","69","72","83","87","91","97","98","101","102","107","46","70","1","9","19","22","24","25","37","38","42","51","52","53","62","65","68","4","8","10","12","17","18","23","33","35","40","41","44","55","57","58","64","71","77","80","81","85","88","92","96","99","100","103","104","109","110","112")
	my_colors<-c("darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","violetred4","violetred4","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4")
	metadata <- sample_data(phy_percentuali_reduced)
	metadata$ID_original <- factor(metadata$ID_original) #, levels = new_order )
	sample_data(phy_percentuali_reduced) <- metadata
	P <- plot_bar(phy_percentuali_reduced,  x="ID_original", fill="IBD") + facet_grid(~IBD,scales="free_x")  + theme(axis.text.x = element_text(angle = 45, hjust = 1)) +facet_wrap(~IBD,scales="free_x") +ggtitle(taxa)
	#P <- plot_bar(phy_percentuali_reduced,  x="ID_original", fill="IBD", facet_grid=~IBD) + facet_grid(scales="free_x") + theme(axis.text.x = element_text(angle = 45, hjust = 1)) +facet_wrap(~IBD) +ggtitle(genus)
	#P <- plot_bar(phy_percentuali_reduced,  x="ID_original", fill="IBD", facet_grid=~IBD) + facet_grid(scales="free_x") 
	Save.graph( Graph=P, OutDir="Immagini", Name=taxa, Name_prefix=paste(taxa,"IBD_0_1_") )
	
	# adesso possiamo provare a fare a stampare in funzione delle varie scale montreal, riducendo ai soli pazienti
	phy_percentuali_reduced_patients <- subset_samples( phy_percentuali_reduced, IBD == "1" )
	my_colors_due<-c("darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","darkgreen","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4","violetred4")

	P1 <- plot_bar(phy_percentuali_reduced_patients, x="ID_original", fill="Montreal_MDC_localizzazione", facet_grid=~Montreal_MDC_localizzazione) + facet_grid(scales="free_x") +
		theme(axis.text.x = element_text(angle = 45, hjust = 1))+ ggtitle(taxa) #+ facet_wrap(~Montreal_MDC_localizzazione, scales="free_x")
        
	P2 <- plot_bar(phy_percentuali_reduced_patients, x="ID_original", fill="Montreal_MDC_Eta", facet_grid=~Montreal_MDC_Eta) + facet_grid(scales="free_x") +
		theme(axis.text.x = element_text(angle = 45, hjust = 1)) + ggtitle(taxa) #+facet_wrap(~Montreal_MDC_Eta,scales="free_x")
        
	P3 <- plot_bar(phy_percentuali_reduced_patients, x="ID_original", fill="Montreal_MDC_Fenotipo", facet_grid=~Montreal_MDC_Fenotipo) + facet_grid(scales="free_x") +
		theme(axis.text.x = element_text(angle = 45, hjust = 1))+ ggtitle(taxa)#+facet_wrap(~Montreal_MDC_Fenotipo, scales="free_x")
        
	
	Save.graph( Graph=P1, OutDir="Immagini", Name=taxa, Name_prefix=paste(genus,"IBD_Montreal_Loc_"))
	Save.graph( Graph=P2, OutDir="Immagini", Name=taxa, Name_prefix=paste(genus,"IBD_Montreal_Eta_"))
	Save.graph( Graph=P3, OutDir="Immagini", Name=taxa, Name_prefix=paste(genus,"IBD_Montreal_Fen_"))
	
	
	cat("\n")
    }
}

In [ ]:
phyG_percentuali

In [ ]:
generi <- as.vector(table$Family)
#generi <- c("\\[Eubacterium\\] ruminantium group","\\[Eubacterium\\] xylanophilum group","\\[Eubacterium\\] ventriosum group")
for(genus in generi){
    print(genus)
    if (genus != "uncultured" && genus != "gut metagenome"){
        righe <- grepl(genus,tassonomiaG$Family,fixed=FALSE,ignore.case=TRUE)
        tassonomia_genus <- tassonomiaG[ righe , ]
        barplotting_variables(genus, tassonomia_genus,level,phyG_percentuali)
        } 
}

In [ ]:
generi <- as.vector(table$Genus)
df <- data.frame(Genus=character(), Pvalue=numeric(), Chisquared = numeric(), df=numeric())
for(genus in generi){
    if (genus== "Escherichia-Shigella" | genus== "Enterococcus" | genus =="Lactobacillus"){
        print(genus)
        righe <- grepl(genus,tassonomiaG$Genus,fixed=FALSE,ignore.case=TRUE)
        tassonomia_genus <- tassonomiaG[ righe , ]
        for (r in 1:nrow(tassonomia_genus)){
            taxa <- as.character(tassonomia_genus[r,]$level)
            taxa <- gsub('\\.', '', taxa)
            taxa <- gsub(':', '', taxa)
            ID_taxa <- rownames(tassonomia_genus[r,])
            cat("Analisi della tassonomia: ",taxa," con ID: ",ID_taxa,"\n")
        
            # analisi per valutare la concentrazione tra IBD = 0 vs IBD = 1
            phy_percentuali_reduced <- take_taxa(phyG_percentuali, ID_taxa)
            phy_percentuali_reduced <- subset_samples( phy_percentuali_reduced , Montreal_MDC_localizzazione != "" )
            #test_differential_abundance_Kruskal(phy_percentuali_reduced, Montreal_MDC_localizzazione, compare = NULL,
            #block = , p.adjust.method = "fdr")
            phy_df <- as.data.frame(t(as.data.frame(otu_table(phy_percentuali_reduced))))
        
            phy_df$Montreal_MDC_localizzazione <- sample_data(phy_percentuali_reduced)$Montreal_MDC_localizzazione
        
            colnames(phy_df) <- c("Genus","Montreal_MDC_localizzazione")
        
            res <- kruskal.test(Genus ~ Montreal_MDC_localizzazione, data = phy_df )
            row <- data.frame(c(genus),c(res$p.value),c(res$statistic),c(res$parameter))
            colnames(row) <- c("Genus","Pvalue","Chisquared","df")
            row.names(row) <- c()
            df <- rbind(df,row)
            
            }
        
        } 
}

In [ ]:
write.csv(df,"Montreal_MDC_localizzazione_kruskal.test.csv")

In [ ]:
d <- as.data.frame(otu_table(phy))

d <- t(d[rownames(table),])

tax <- tax_table(phy)[rownames(table),level]

merged <- merge(t(d), tax, by="row.names",all.x=TRUE )

d1 <- sample_data(phy)[,"IBD"]

d <-merged

d$Genus <- NULL

row.names(d)<-merged$Genus

d$Row.names <- NULL

d <- t(d)

d <- as.data.frame(merge(d,d1,by="row.names",all.x=TRUE))

temp <- d

d$Row.names <- NULL

row.names(d) <- temp$Row.names

#write.csv(df,"few_analysis.csv")

In [ ]:
#range01 <- function(x){(x-min(x))/(max(x)-min(x))}

#d$value <- range01(d$value)

d <- melt(data = d, Genus = "Genus", measure.vars = colnames(d)[1:9])

families <- t(tax_table(phy)[,"Genus"])




png("few_analysis.png")
p <-  ggplot(d, aes(x=variable, y=value, fill=IBD)) + 
    geom_boxplot()+facet_wrap(vars(variable), scales="free")
print(p)
dev.off()
p

In [ ]:
#seleziono solo i pazienti che hanno il clostridium
phyG_innocuum <- subset_samples(phyF, ID_original == "1" | ID_original == "22" | ID_original == "37" | ID_original == "38" | ID_original == "57" |
    ID_original == "58" | ID_original == "77" | ID_original == "80" | ID_original == "81" | ID_original == "85" | ID_original == "103" |
    ID_original == "109" | ID_original == "110" | ID_original == "8" | ID_original == "10" | ID_original == "12" | ID_original == "17" |
    ID_original == "33" | ID_original == "35")

df_innocuum <- as.data.frame(sample_data(phyG_innocuum)$Montreal)
values <- c(0.045932135270138376, 0.23414329569696654, 0.7023875624652971, 0.05609835912299565, 0.2563267963219139, 0.017537321612556722, 0.1222759096736022, 0.04022028339830456, 0.3533084073680239, 0.05528193788320434, 0.25521259690855685, 0.03292045594831488, 0.1080580426057425, 0.07802678919762453, 0.14455862480751705, 0.1788508830762352, 0.5090977656264731, 0.032783953744821626, 0.14086367037901132)
colnames(df_innocuum)<-"Montreal"
row.names(df_innocuum)<- sample_data(phyG_innocuum)$ID_original
df_innocuum$percent_reads<- values
df_innocuum <- df_innocuum[order(df_innocuum$percent_reads),]

#seleziono i pazienti che non hanno il clostridium
phyG_NO_innocuum <- subset_samples(phyF, ID_original == '9'| ID_original == '19'| ID_original == '24'| ID_original == '25'| ID_original == '42'| ID_original == '51'| ID_original == '52'| ID_original == '62'| ID_original == '65'| ID_original == '68'| ID_original == '40'| ID_original == '41'| ID_original == '44'| ID_original == '53'| ID_original == '55'| ID_original == '64'| ID_original == '71'| ID_original == '88'| ID_original == '92'| ID_original == '96'| ID_original == '99'| ID_original == '100'| ID_original == '104'| ID_original == '112'| ID_original == '4'| ID_original == '18'| ID_original == '23')

df_NO_innocuum <- as.data.frame(sample_data(phyG_NO_innocuum)$Montreal)
colnames(df_NO_innocuum)<-"Montreal"
row.names(df_NO_innocuum)<- sample_data(phyG_NO_innocuum)$ID_original

In [ ]:
library("ggplot2")
theme_set(theme_bw())
scale_fill_discrete <- function(palname = "Set1", ...) {
    scale_fill_brewer(palette = palname, ...)
}
# Phylum order
x = tapply(sigtab$log2FoldChange, sigtab$Phylum, function(x) max(x))
x = sort(x, TRUE)
sigtab$Phylum = factor(as.character(sigtab$Phylum), levels=names(x))
ggplot(sigtab, aes(x=Family, y=log2FoldChange, color=Phylum)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5))

In [ ]:
# Genus order
x = tapply(sigtab$log2FoldChange, sigtab$Genus, function(x) max(x))
x = sort(x, TRUE)
sigtab$Genus = factor(as.character(sigtab$Genus), levels=names(x))
ggplot(sigtab, aes(x=Genus, y=log2FoldChange, color=Phylum)) + geom_point(size=6) + 
  theme(axis.text.x = element_text(angle = -90, hjust = 0, vjust=0.5))

In [ ]:
?merge